In [1]:
%load_ext autoreload
%autoreload 2

root = r"C:\Users\zwecher\Documents\mygit\new_predictor"
#predictor_path  = r"C:\Users\zwecher\Documents\mygit\Indoors_Capabilities\predictor"
#model_path = r"C:\Users\zwecher\Documents\mygit\Indoors_Capabilities\models"

from importlib import reload  # Not needed in Python 2
import logging
reload(logging)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import models
from tensorflow.keras import backend as K 

from tensorflow import keras
import sys, os
sys.path.insert(0,root)
#from predictor import predictor, config, restore
import utils
from predictor import buildings_predictor as bp
from predictor import batch_manager
#from predictor import full_model_predictor as fmp
utils.set_log()
os.chdir(r"C:\Users\zwecher\Documents\mygit\new_predictor\predictor")

C:\Users\zwecher\AppData\Local\Continuum\anaconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\zwecher\AppData\Local\Continuum\anaconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\zwecher\AppData\Local\Continuum\anaconda3\envs\keras\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 


2019-11-18 12:29:02,372 [INFO: __init__.py 20] path: ..\logs


In [2]:
#K.clear_session()
model = bp.get_model()


2019-11-18 12:29:12,300 [INFO: buildings_predictor.py 14] Tensor("dense_target:0", shape=(None, None), dtype=float32)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

2019-11-18 12:29:12,300 [WARNING: deprecation.py 323] From C:\Users\zwecher\AppData\Local\Continuum\anaconda3\envs\keras\lib\site-packages\tensorflow\python\ops\nn_impl.py:182: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
my_input (Conv2D)            (None, 56, 54, 7)         350       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 26, 7)         0         
_________________________________

In [3]:
model.get_config()

{'name': 'sequential',
 'layers': [{'class_name': 'Conv2D',
   'config': {'name': 'my_input',
    'trainable': True,
    'batch_input_shape': (None, 62, 60, 1),
    'dtype': 'float32',
    'filters': 7,
    'kernel_size': (7, 7),
    'strides': (1, 1),
    'padding': 'valid',
    'data_format': 'channels_last',
    'dilation_rate': (1, 1),
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'MaxPooling2D',
   'config': {'name': 'max_pooling2d',
    'trainable': True,
    'dtype': 'float32',
    'pool_size': (4, 4),
    'padding': 'valid',
    'strides': (2, 2),
    'data_format': 'channels_last'}},
  {'class_name': 'Conv2D',
   'config': {'name': 'conv2d',
    'traina

In [6]:
metadata_df = batch_manager.create_metadata_df()
bm = batch_manager.BatchManager(metadata_df,test_num=256)
x_t,y = bm.get_test_batch()
func = utils.get_inermediate_layers_func(model,["dense"])
for i in range(5):
    x,y = bm.get_next_train_batch(256)
    model.train_on_batch(x, y)
    if i%1 == 0:
        loss = model.evaluate(x,y)
        print(loss)
        print(func(x))

256/256 [==============================] - 0s 206us/sample - loss: 0.2384
0.23839751072227955
[array([[6.608182 , 6.6220417, 6.622451 , ..., 6.616274 , 6.6516986,
        6.6447616],
       [6.558717 , 6.6185427, 6.5700436, ..., 6.5830817, 6.5723557,
        6.590183 ],
       [6.387796 , 6.410137 , 6.406206 , ..., 6.366886 , 6.4334664,
        6.423634 ],
       ...,
       [6.7651696, 6.8632812, 6.826542 , ..., 6.828661 , 6.919186 ,
        6.858289 ],
       [6.6097836, 6.6638145, 6.681063 , ..., 6.65318  , 6.6505356,
        6.679927 ],
       [6.4513783, 6.522175 , 6.5323787, ..., 6.550554 , 6.4975514,
        6.5658617]], dtype=float32)]
256/256 [==============================] - 0s 199us/sample - loss: 0.2382
0.23816325515508652
[array([[6.86973  , 6.955236 , 6.9578056, ..., 6.9814634, 6.9069333,
        6.9452996],
       [6.8050413, 6.917578 , 6.9783287, ..., 6.8651085, 6.8862963,
        6.906159 ],
       [6.9975023, 7.0374537, 7.0467424, ..., 6.9983993, 7.06461  ,
        7

In [ ]:
bp.train_model(model,3)

Epoch 1/3
880/976 [==========================>...] - ETA: 4:44 - loss: 0.2087